In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "write your openai key"

In [3]:
!pip install langchain chromadb faiss-cpu openai tiktoken langchain_openai langchain-community wikipedia

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from langchain_community.retrievers import WikipediaRetriever

In [5]:
wiki_retriver  = WikipediaRetriever(top_k_results=2,lang="en")

In [6]:
query = "who is the mahatama gandhi"

docs = wiki_retriver.invoke(query)

In [7]:
for i,doc in enumerate(docs):
  print(f"\n ------ result {i} -------\n")
  print(doc.page_content)


 ------ result 0 -------

Mohandas Karamchand Gandhi (2 October 1869 – 30 January 1948) was an Indian lawyer, anti-colonial nationalist, and political ethicist who employed nonviolent resistance to lead the successful campaign for India's independence from British rule. He inspired movements for civil rights and freedom across the world. The honorific Mahātmā (from Sanskrit, meaning great-souled, or venerable), first applied to him in South Africa in 1914, is used worldwide.
Born and raised in a Hindu family in coastal Gujarat, Gandhi was trained in the law at the Inner Temple in London and was called to the bar at the age of 22. After two uncertain years in India, where he was unable to start a successful law practice, Gandhi moved to South Africa in 1893 to represent an Indian merchant in a lawsuit. He went on to live in South Africa for the next 21 years. Here, Gandhi raised a family and first employed nonviolent resistance in a campaign for civil rights. In 1915, aged 45, he retur

In [8]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

In [9]:
# Step 1: Your source documents
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [10]:
embedding_models = OpenAIEmbeddings()
vector_embedding = Chroma.from_documents(
    documents= documents,
    embedding = embedding_models,
    collection_name = "my_collection"

)

In [11]:
vec_reterivers = vector_embedding.as_retriever(search_kwargs ={"k": 2})

In [12]:
query = "what does open ai do"

results = vec_reterivers.invoke(query)

In [13]:
for i, result in enumerate(results):
  print(f"------ result{1}------- \n")
  print(f"{result.page_content}")

------ result1------- 

OpenAI provides powerful embedding models.
------ result1------- 

Embeddings convert text into high-dimensional vectors.


In [14]:
result_similarity = vector_embedding.similarity_search(query, k =2)

In [15]:
'''
maximal margunal revelence
it is retrieval stretegy that
that pick results that are not only relevent to the query but also different than each other


the problem statement could be
lets this be a DB
D1: Gandhi was born on 2 October 1869 in Porbandar.
D2: Mahatma Gandhi was born in Gujarat in 1869.
D3: Gandhi led the Salt March in 1930.
D4: Gandhi pioneered non-violent resistance (Satyagraha).
D5: Satyagraha influenced global civil rights movements.
 now if we do a normal simialrity search based search then we will get
with k = 3
 D1: Gandhi was born on 2 October 1869 in Porbandar.
D2: Mahatma Gandhi was born in Gujarat in 1869
D3: Gandhi led the Salt March in 1930.

THE FIRST TWO outputs are same only
to tackle this we use maximal marginal relevance



'''



'\nmaximal margunal revelence \nit is retrieval stretegy that\nthat pick results that are not only relevent to the query but also different than each other \n\n\nthe problem statement could be \nlets this be a DB\nD1: Gandhi was born on 2 October 1869 in Porbandar.\nD2: Mahatma Gandhi was born in Gujarat in 1869.\nD3: Gandhi led the Salt March in 1930.\nD4: Gandhi pioneered non-violent resistance (Satyagraha).\nD5: Satyagraha influenced global civil rights movements.\n now if we do a normal simialrity search based search then we will get\nwith k = 3\n D1: Gandhi was born on 2 October 1869 in Porbandar.\nD2: Mahatma Gandhi was born in Gujarat in 1869\nD3: Gandhi led the Salt March in 1930.\n\nTHE FIRST TWO outputs are same only \nto tackle this we use maximal marginal relevance\n\n\n\n'

In [16]:
docs_mmr = [
Document(page_content = "Gandhi was born on 2 October 1869 in Porbandar"),
Document(page_content = "Mahatma Gandhi was born in Gujarat in 1869."),
Document(page_content = "Gandhi led the Salt March in 1930."),
Document(page_content= "Gandhi pioneered non-violent resistance (Satyagraha)"),
Document(page_content = "Satyagraha influenced global civil rights movements.")
]

In [17]:
from langchain_community.vectorstores import FAISS
embedding_models = OpenAIEmbeddings()
vector_store = FAISS.from_documents(
    documents = docs_mmr,
    embedding = embedding_models,

)


In [18]:
retriver_mmr = vector_store.as_retriever(
    search_type = "mmr",
    search_kwargs = {"k": 3, "lambda_mult": 0.5} # here the mult range is 0 - 1 with decreasing order of difference
    # meaning this will 0 will give highest difference and 1 will give lowest difference
)


In [19]:
results = retriver_mmr.invoke("who is gandhi")
results
# here it can be seen that every output is diferent from another

[Document(id='a4c5ee26-a437-4893-9381-cd0fe3424ee2', metadata={}, page_content='Gandhi was born on 2 October 1869 in Porbandar'),
 Document(id='93d4f084-4752-4e49-9705-2d4434e4796b', metadata={}, page_content='Gandhi pioneered non-violent resistance (Satyagraha)'),
 Document(id='37f3c1bc-ee31-4906-b588-8f4c253eebda', metadata={}, page_content='Gandhi led the Salt March in 1930.')]

In [20]:
'''
multiquery retirvers
some times the query can be very vauge and broad and answering it can be required to give answer with many different context
for example

query - how to be healthy ?

now here is user asking excercise tips
or dietery advice
or talking about mental heatlhy
or wanted to grow muscles ?

so to give an answer based upon allot of differnt context multi query retriever can be used

and here is how it works

your query is conveerted into many relevent query by an llm and then k documnet related to related to those
query are found in the document and then merged and return to user depending upon k

'''

'\nmultiquery retirvers \nsome times the query can be very vauge and broad and answering it can be required to give answer with many different context\nfor example\n\nquery - how to be healthy ?\n\nnow here is user asking excercise tips\nor dietery advice \nor talking about mental heatlhy \nor wanted to grow muscles ?\n\nso to give an answer based upon allot of differnt context multi query retriever can be used\n\nand here is how it works\n\nyour query is conveerted into many relevent query by an llm and then k documnet related to related to those \nquery are found in the document and then merged and return to user depending upon k \n\n'

In [21]:
!pip install langchain_classic

In [22]:
!pip install -U langchain

In [23]:


all_docs = [
    # 🏃 Exercise
    Document(
        page_content="Regular walking boosts heart health and improves cardiovascular endurance.",
        metadata={"source": "H1"}
    ),
    Document(
        page_content="Strength training helps build muscle mass and improves bone density.",
        metadata={"source": "H2"}
    ),
    Document(
        page_content="Aerobic exercises like running and cycling improve stamina and lung capacity.",
        metadata={"source": "H3"}
    ),
    Document(
        page_content="Daily physical activity helps regulate blood sugar and metabolism.",
        metadata={"source": "H4"}
    ),

    # 🥗 Diet & Nutrition
    Document(
        page_content="Consuming leafy greens and fruits provides essential vitamins and antioxidants.",
        metadata={"source": "H5"}
    ),
    Document(
        page_content="A balanced diet with proteins, fats, and carbohydrates supports overall health.",
        metadata={"source": "H6"}
    ),
    Document(
        page_content="Adequate protein intake supports muscle recovery and tissue repair.",
        metadata={"source": "H7"}
    ),
    Document(
        page_content="Drinking sufficient water throughout the day helps maintain energy levels.",
        metadata={"source": "H8"}
    ),

    # 🧠 Mental Health
    Document(
        page_content="Mindfulness practices help reduce anxiety and improve emotional regulation.",
        metadata={"source": "H9"}
    ),
    Document(
        page_content="Meditation lowers stress hormones and improves focus and clarity.",
        metadata={"source": "H10"}
    ),
    Document(
        page_content="Managing stress effectively is essential for long-term mental well-being.",
        metadata={"source": "H11"}
    ),

    # 😴 Sleep & Recovery
    Document(
        page_content="Deep sleep is crucial for cellular repair and emotional balance.",
        metadata={"source": "H12"}
    ),
    Document(
        page_content="Sleep deprivation negatively impacts immune function and cognition.",
        metadata={"source": "H13"}
    ),
    Document(
        page_content="Consistent sleep schedules improve recovery and hormone regulation.",
        metadata={"source": "H14"}
    ),

    # 🌱 Lifestyle
    Document(
        page_content="Spending time outdoors improves mood and supports vitamin D production.",
        metadata={"source": "H15"}
    ),
    Document(
        page_content="Maintaining strong social connections supports emotional health.",
        metadata={"source": "H16"}
    ),
    Document(
        page_content="Healthy daily routines contribute to long-term well-being.",
        metadata={"source": "H17"}
    ),

    # ❌ Irrelevant / Noise (to show filtering effect)
    Document(
        page_content="Solar panels convert sunlight into electrical energy for homes.",
        metadata={"source": "I1"}
    ),
    Document(
        page_content="Python is a popular programming language known for simplicity.",
        metadata={"source": "I2"}
    ),
    Document(
        page_content="Black holes warp spacetime due to intense gravitational forces.",
        metadata={"source": "I3"}
    )
]


In [24]:
vectorstore = FAISS.from_documents(documents=all_docs, embedding=embedding_models)

In [25]:
# Create retrievers
similarity_retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [31]:
from langchain_classic.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=retriever,
    llm=ChatOpenAI(model="gpt-3.5-turbo")
)
'''
multiquery have been removed from langchain and now is avilable in langchain_classic
'''

'\nmultiquery have been removed \n'

In [32]:
# Query
query = "How to improve energy levels and maintain balance?"

In [33]:
# Retrieve results
similarity_results = similarity_retriever.invoke(query)
multiquery_results= multiquery_retriever.invoke(query)

In [34]:
for i, doc in enumerate(similarity_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)

print("*"*150)

for i, doc in enumerate(multiquery_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Drinking sufficient water throughout the day helps maintain energy levels.

--- Result 2 ---
Managing stress effectively is essential for long-term mental well-being.

--- Result 3 ---
Meditation lowers stress hormones and improves focus and clarity.

--- Result 4 ---
Healthy daily routines contribute to long-term well-being.

--- Result 5 ---
Daily physical activity helps regulate blood sugar and metabolism.
******************************************************************************************************************************************************

--- Result 1 ---
Drinking sufficient water throughout the day helps maintain energy levels.

--- Result 2 ---
Managing stress effectively is essential for long-term mental well-being.

--- Result 3 ---
Healthy daily routines contribute to long-term well-being.

--- Result 4 ---
Deep sleep is crucial for cellular repair and emotional balance.

--- Result 5 ---
Meditation lowers stress hormones and improves focus an

In [1]:
'''
context compression retriever


Retrievers often return long documents, but only a small part is actually useful for the question.

Context compression cuts away the unnecessary parts and keeps only the relevant lines.

This saves tokens and helps the AI give more accurate answers.



🎯 Example scenario

Corpus:
Full-length research papers (PDFs) on lung cancer (10–20 pages each)

User question:

“Which genes are involved in LUAD → LUSC transition?”

🟦 Vanilla Retrieval (no context compression)
Workflow

Query → vector retriever

Top-k documents returned

Entire chunks sent to the LLM

What the LLM receives
• Abstract
• Introduction
• Methods (sequencing, QC, pipelines)
• Results (many unrelated sections)
• Discussion


Even if LUAD/LUSC is mentioned only in 2 paragraphs, the LLM sees everything.

Problems

❌ Wasted tokens
❌ Important signal buried in noise
❌ LLM may:

miss key genes

hallucinate associations

give vague answers

Typical answer

“Several genes and pathways may be involved in lung cancer subtype transitions…”

🟩 Context Compression Retrieval
Workflow

Query → vector retriever (same as above)

Top-k documents returned

Each document is compressed using the query

Only relevant spans are passed to the LLM

What the LLM receives
• “SOX2 upregulation observed during LUAD to LUSC transition…”
• “TP63 and KRT5 expression increased in squamous regions…”
• “Loss of NKX2-1 associated with lineage switching…”


Everything else is discarded.

Benefits

✅ Fewer tokens
✅ High signal
✅ Clear gene-level grounding

Typical answer

“LUAD → LUSC transition is associated with SOX2 and TP63 upregulation, increased KRT5 expression, and loss of NKX2-1…”

'''

'\ncontext compression retriever\n\n\nRetrievers often return long documents, but only a small part is actually useful for the question.\n\nContext compression cuts away the unnecessary parts and keeps only the relevant lines.\n\nThis saves tokens and helps the AI give more accurate answers.\n\n\n\n🎯 Example scenario\n\nCorpus:\nFull-length research papers (PDFs) on lung cancer (10–20 pages each)\n\nUser question:\n\n“Which genes are involved in LUAD → LUSC transition?”\n\n🟦 Vanilla Retrieval (no context compression)\nWorkflow\n\nQuery → vector retriever\n\nTop-k documents returned\n\nEntire chunks sent to the LLM\n\nWhat the LLM receives\n• Abstract\n• Introduction\n• Methods (sequencing, QC, pipelines)\n• Results (many unrelated sections)\n• Discussion\n\n\nEven if LUAD/LUSC is mentioned only in 2 paragraphs, the LLM sees everything.\n\nProblems\n\n❌ Wasted tokens\n❌ Important signal buried in noise\n❌ LLM may:\n\nmiss key genes\n\nhallucinate associations\n\ngive vague answers\n\nTy

In [36]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_classic.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_classic.retrievers.document_compressors import LLMChainExtractor
from langchain_core.documents import Document

In [37]:
# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [38]:
# Create a FAISS vector store from the documents
embedding_model = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embedding_model)

In [39]:
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [40]:
# Set up the compressor using an LLM
llm = ChatOpenAI(model="gpt-3.5-turbo")
compressor = LLMChainExtractor.from_llm(llm)

In [41]:
# Create the contextual compression retriever
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

In [42]:
# Query the retriever
query = "What is photosynthesis?"
compressed_results = compression_retriever.invoke(query)

In [43]:
for i, doc in enumerate(compressed_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)
# it can be scene here that our corpus had big chunks of text but here we get a trimmed within context output and not the whole jargan


--- Result 1 ---
Photosynthesis is the process by which green plants convert sunlight into energy.

--- Result 2 ---
The chlorophyll in plant cells captures sunlight during photosynthesis.
